## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-16-15-06-20 +0000,bbc,Government abandons plans to delay 30 council ...,https://www.bbc.com/news/articles/c70ne31d884o...
1,2026-02-16-15-03-02 +0000,bbc,South African trio charged with Bolt driver's ...,https://www.bbc.com/news/articles/cn9e39zljwxo...
2,2026-02-16-15-02-21 +0000,nypost,Iran diplomat claims ‘we are ready to discuss’...,https://nypost.com/2026/02/16/us-news/iran-dip...
3,2026-02-16-15-00-00 +0000,wsj,Hapag-Lloyd to Buy Israeli Rival Zim for $4.2 ...,https://www.wsj.com/business/deals/hapag-lloyd...
4,2026-02-16-14-58-28 +0000,nyt,"U.S. Deports Nine Migrants in Secret, Ignoring...",https://www.nytimes.com/2026/02/14/world/afric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2479/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
106,trump,19
273,new,9
15,iran,8
206,guthrie,7
83,was,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
147,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...,58
138,2026-02-16-01-20-14 +0000,nypost,Savannah Guthrie tells mom Nancy’s kidnapper ‘...,https://nypost.com/2026/02/15/us-news/savannah...,49
111,2026-02-16-05-39-01 +0000,bbc,Trump's new world order has become real and Eu...,https://www.bbc.com/news/articles/cddn002g6qzo...,43
202,2026-02-15-15-32-07 +0000,bbc,'Trump will be gone in three years': Top Democ...,https://www.bbc.com/news/articles/cjrq2r9y278o...,42
66,2026-02-16-11-00-00 +0000,latimes,Voter trust in U.S. elections drops amid Trump...,https://www.latimes.com/politics/story/2026-02...,36


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
147,58,2026-02-16-00-22-00 +0000,wsj,The U.S. military airlifted on Sunday a miniat...,https://www.wsj.com/politics/national-security...
138,49,2026-02-16-01-20-14 +0000,nypost,Savannah Guthrie tells mom Nancy’s kidnapper ‘...,https://nypost.com/2026/02/15/us-news/savannah...
30,34,2026-02-16-13-44-07 +0000,cbc,"Dana Eden, co-creator of Israeli TV series Teh...",https://www.cbc.ca/news/world/dana-eden-israel...
127,33,2026-02-16-02-33-15 +0000,nypost,Radical SoCal group sets next anti-ICE ‘mass m...,https://nypost.com/2026/02/15/us-news/radical-...
171,27,2026-02-15-19-54-32 +0000,cbc,'It was a gift': French goalie Junca's birthda...,https://www.cbc.ca/sports/olympics/winter/hock...
189,27,2026-02-15-17-00-00 +0000,wsj,Workers Are Afraid AI Will Take Their Jobs. Th...,https://www.wsj.com/lifestyle/careers/ai-knowl...
113,26,2026-02-16-05-31-40 +0000,nypost,Maniac who allegedly pushed woman onto NYC sub...,https://nypost.com/2026/02/16/us-news/maniac-w...
175,24,2026-02-15-19-32-22 +0000,nypost,Kim Jong-Un’s 13-year-old heiress could face d...,https://nypost.com/2026/02/15/world-news/kim-j...
167,24,2026-02-15-20-15-29 +0000,nypost,Ex-Prince Andrew ghosting Oversight Committee’...,https://nypost.com/2026/02/15/us-news/ex-princ...
74,22,2026-02-16-10-30-00 +0000,wsj,Americans offer plenty of reasons for pessimis...,https://www.wsj.com/personal-finance/economic-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
